# Notebook estimates strikes and number of contracts for a put selling strategy on equity indices such as S&P 500. 

Algorithm is drawn from  JUREK, J. W. and STAFFORD, E. (2015), The Cost of Capital for Alternative Investments. The Journal of Finance, [Cost_of_Capital](https://www.hbs.edu/faculty/Publication%20Files/Cost%20of%20Capital%20for%20Alternative%20Investments_57a4f444-65fa-4f0c-b51a-116408f1dab9.pdf)

 - Requires an open session of either IB Gateway or TWS from Interactive Brokers

In [2]:
from IPython.display import display_html, HTML
import pyfolio as pf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nest_asyncio
from time import time
import plistlib
import sys
opt_path = '/Users/ebellord/PycharmProjects/opt_trade'
if  opt_path not in sys.path:
    sys.path.append(opt_path)

from option_daily_prod import OptionMarket, SpxOptionAsset, RSL2OptionAsset
from spx_data_update import illiquid_equity
from option_simulation import OptionSimulation, OptionTrades
# nest_asyncio.apply()

In [3]:
illiquid_capital = illiquid_equity(discount=0.5)
%time spx_asset = OptionMarket(SpxOptionAsset(trading_class='SPXW'))
# rsl_asset = OptionMarket(RSL2OptionAsset())

CPU times: user 377 ms, sys: 45.8 ms, total: 423 ms
Wall time: 1.34 s


In [5]:
z_score = np.array([-0.5, -1, -1.5])
leverage = np.array(range(1, 4, 1))
# num_expiries = 4
num_expiries = [9, 10, 11, 12, 13, 14, 15]
spx = True
if spx:
    %time trd_choice = spx_asset.form_trade_choice(z_score, num_expiries, 'P')
else:
    trd_choice = rsl_asset.form_trade_choice(z_score, num_expiries, 'P')    
last_price = trd_choice.spot

print('Last Price: {}'.format(last_price))
print('Sigma: {}'.format(trd_choice.sigma))
account_number = trd_choice.account_value[0][0]
account_value = float(trd_choice.account_value[0][2])

capital_at_risk = account_value + illiquid_capital
print('Account {} value is: ${:,.0f}'.format(account_number, account_value))
print('Illiquid Equity after haircut is ${:,.0f}'.format(float(illiquid_capital)))
print('Capital at Risk ${:,.0f}'.format(float(capital_at_risk)))

CPU times: user 55 ms, sys: 5.84 ms, total: 60.9 ms
Wall time: 14.7 s
Last Price: 2945.64
Sigma: 12.87
Account U2463043 value is: $2,406,840
Illiquid Equity after haircut is $560,375
Capital at Risk $2,967,215


# Strikes are writen according to:
$$K(Z)= S.exp\Big\{\big( r_{f}(\tau) - q(\tau) + \frac{\sigma^{2}}{2}\big).\tau + \sigma(\tau).\sqrt{\tau}.Z\Big\}$$

Where Z is Z-score that ensures constant delta exposure 

In [6]:
def format_index(df):
    df = df.set_index(df.index.strftime('%Y.%m.%d'))
    return df

def df_styler(df, table_name=None):
    df = format_index(df)
    df_styler_out = df.style.set_table_attributes("style='display:inline'").set_caption(table_name)
    return df_styler_out

def super_styler(dfs, table_names, format_str):
    df_style_list = [df_style.format(format_str) for df_style in 
                      [df_styler(df, tbl_name) for df, tbl_name  in zip(dfs, table_names)]]
    df_style_list = [item._repr_html_() for item in df_style_list]
    str_out = ''.join(df_style_list)
    return str_out

def title_display(title):
    if spx_asset.option_asset.trading_class == 'SPXW':
        title = title + ' - Weeklys'
    display(HTML('<h1>' + title + '</h1>'))


title_display('Trade Grid')
display_html(df_styler(trd_choice.strike_grid, 'Strike Grid')._repr_html_() + 
             df_styler(trd_choice.premium_grid, 'Premium Grid')._repr_html_() + 
             df_styler(trd_choice.prices_grid, 'Prices Grid')._repr_html_(), raw=True) 

prct_decline =  format_index(trd_choice.pct_otm_grid(last_price))
format_dict = {x: '{:.2%}' for x in prct_decline.to_dict()}

title_display('Strike Moneyness')
display(prct_decline.style.format(format_dict).set_caption('Strike % OTM'))

leverage_table_names = ['Leverage: ' + str(lev) for lev in leverage]
premium_list =  [(trd_choice.premium_grid * contractTable.round() * 100) \
                 for contractTable in trd_choice.option_lots(leverage, capital_at_risk)]

title_display('Premium')
display_html(super_styler(premium_list, leverage_table_names, '${:,.0f}'), raw=True)

title_display('Premium as % of capital at risk')
premium_list_pct = [df / capital_at_risk for df in premium_list]
display_html(super_styler(premium_list_pct, leverage_table_names, '{:.2%}'), raw=True)
contract_table_list = trd_choice.option_lots(leverage, capital_at_risk)

title_display('Contract to Trade')
display_html(super_styler(contract_table_list, leverage_table_names, '{:.0f}'), raw=True)

title_display('Margin')
margin_dlr_list = [trd_choice.margin(last_price) * df for df in trd_choice.option_lots(leverage, capital_at_risk)]
display_html(super_styler(margin_dlr_list, leverage_table_names, '${:,.0f}'), raw=True)

title_display('Notional $ Exposure')
notional_dlr=[df * trd_choice.strike_grid * 100 for df in trd_choice.option_lots(leverage, capital_at_risk)]
display_html(super_styler(notional_dlr, leverage_table_names, '${:,.0f}'), raw=True)

,-0.5,-1.0,-1.5
2019.05.28,2900,2850,2805
2019.05.29,2900,2850,2800
2019.05.31,2895,2845,2795
2019.06.03,2895,2840,2790
2019.06.07,2890,2835,2780
2019.06.14,2885,2825,2765
2019.06.21,2880,2815,2750
,-0.5,-1.0,-1.5
2019.05.28,29,17.45,11.55
2019.05.29,30,18.35,11.8


,-0.5,-1.0,-1.5
2019.05.28,-1.55%,-3.25%,-4.77%
2019.05.29,-1.55%,-3.25%,-4.94%
2019.05.31,-1.72%,-3.42%,-5.11%
2019.06.03,-1.72%,-3.59%,-5.28%
2019.06.07,-1.89%,-3.76%,-5.62%
2019.06.14,-2.06%,-4.10%,-6.13%
2019.06.21,-2.23%,-4.44%,-6.64%


,-0.5,-1.0,-1.5
2019.05.28,"$29,000","$17,450","$12,705"
2019.05.29,"$30,000","$18,350","$12,980"
2019.05.31,"$30,500","$21,230","$14,080"
2019.06.03,$0,$0,$0
2019.06.07,"$33,600","$23,650","$15,785"
2019.06.14,"$36,450","$25,905","$17,325"
2019.06.21,"$39,150","$27,995","$18,810"
,-0.5,-1.0,-1.5
2019.05.28,"$60,900","$36,645","$24,255"
2019.05.29,"$63,000","$38,535","$24,780"


,-0.5,-1.0,-1.5
2019.05.28,0.98%,0.59%,0.43%
2019.05.29,1.01%,0.62%,0.44%
2019.05.31,1.03%,0.72%,0.47%
2019.06.03,0.00%,0.00%,0.00%
2019.06.07,1.13%,0.80%,0.53%
2019.06.14,1.23%,0.87%,0.58%
2019.06.21,1.32%,0.94%,0.63%
,-0.5,-1.0,-1.5
2019.05.28,2.05%,1.23%,0.82%
2019.05.29,2.12%,1.30%,0.84%


,-0.5,-1.0,-1.5
2019.05.28,10,10,11
2019.05.29,10,10,11
2019.05.31,10,11,11
2019.06.03,10,10,11
2019.06.07,10,11,11
2019.06.14,10,11,11
2019.06.21,10,11,11
,-0.5,-1.0,-1.5
2019.05.28,21,21,21
2019.05.29,21,21,21


,-0.5,-1.0,-1.5
2019.05.28,"$572,488","$510,938","$506,042"
2019.05.29,"$573,488","$511,838","$500,817"
2019.05.31,"$568,988","$558,567","$496,417"
2019.06.03,"$538,488","$483,488","$476,837"
2019.06.07,"$567,088","$549,987","$481,622"
2019.06.14,"$564,938","$541,242","$466,662"
2019.06.21,"$562,638","$532,332","$451,647"
,-0.5,-1.0,-1.5
2019.05.28,"$1,202,225","$1,072,970","$966,080"
2019.05.29,"$1,204,325","$1,074,860","$956,105"


,-0.5,-1.0,-1.5
2019.05.28,"$2,900,000","$2,850,000","$3,085,500"
2019.05.29,"$2,900,000","$2,850,000","$3,080,000"
2019.05.31,"$2,895,000","$3,129,500","$3,074,500"
2019.06.03,"$2,895,000","$2,840,000","$3,069,000"
2019.06.07,"$2,890,000","$3,118,500","$3,058,000"
2019.06.14,"$2,885,000","$3,107,500","$3,041,500"
2019.06.21,"$2,880,000","$3,096,500","$3,025,000"
,-0.5,-1.0,-1.5
2019.05.28,"$6,090,000","$5,985,000","$5,890,500"
2019.05.29,"$6,090,000","$5,985,000","$5,880,000"


In [ ]:
spx_asset.option_asset.trading_class